In [ ]:
#LIBRAIRIES IMPORTATION
###############################################

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#DATA IMPORTATION
###############################################

#Import the data from the data folder
#in the data folder there is two folder one is Testing and another is Training
#in each folder there is 4 other folder one is glioma_tumor,meningioma_tumor,no_tumor and pituitary_tumor
#in each folder there is images of the tumor in a jpg format
#we are going to import the data from the training folder
#we are going to import the data from the testing folder

CATEGORIES = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
TRAINING_DIR = "H:/Desktop/Machine Learning/CS584-TumorsBrain/data/Training/"
TESTING_DIR = "H:/Desktop/Machine Learning/CS584-TumorsBrain/data/Testing/"
IMG_SIZE = 256


training_data = []
testing_data = []


#This function is going to read the image and convert it into a numpy array
#and then resize the image to 256*256
#and then append the image and the class number to the training_data list if the parameter train is true else append to the testing_data list
def create_data(train = True):
    for category in CATEGORIES:
        if train:
            path = os.path.join(TRAINING_DIR, category)
        else:
            path = os.path.join(TESTING_DIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                if train:
                    training_data.append([new_array, class_num])
                else:
                    testing_data.append([new_array, class_num])
            except Exception as e:
                pass


#We are going to call the create_data function to create the training data and testing data
create_data(train = True)
create_data(train = False)




In [ ]:
#DATA VISUALIZATION
###############################################

#We are going to visualize the data
#We are going to visualize the training data
#We are going to visualize the testing data

#Visualize the sample number of each class in the training data
def visualize_data(train = True):
    glioma_tumor = 0
    meningioma_tumor = 0
    no_tumor = 0
    pituitary_tumor = 0
    data_set = training_data if train else testing_data
    for features, label in data_set:
        if label == 0:
            glioma_tumor += 1
        elif label == 1:
            meningioma_tumor += 1
        elif label == 2:
            no_tumor += 1
        elif label == 3:
            pituitary_tumor += 1
    data_set_name = "TRAINING" if train else "TESTING"
    print(data_set_name + " DATA SET:")
    print("glioma_tumor: ", glioma_tumor)
    print("meningioma_tumor: ", meningioma_tumor)
    print("no_tumor: ", no_tumor)
    print("pituitary_tumor: ", pituitary_tumor)
    print("Total number of images: ", glioma_tumor + meningioma_tumor + no_tumor + pituitary_tumor)
    plt.bar(["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"], [glioma_tumor, meningioma_tumor, no_tumor, pituitary_tumor])
    if train:
        plt.title("Training Data")
    else:
        plt.title("Testing Data")
    plt.show()
    print("")


#We are going to call the visualize_data function to visualize the training data and testing data
visualize_data(train = True)
visualize_data(train = False)


#We will show one image from each class in the training data
def show_image(train = True):
    data_set = training_data if train else testing_data
    data_set_name = "TRAINING" if train else "TESTING"
    print(data_set_name + " DATA SET:")
    #make a 4*4 grid and big size of each image
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    #glioma_tumor
    for features, label in data_set:
        if label == 0:
            axs[0, 0].imshow(features, cmap = "gray")
            axs[0, 0].set_title("glioma_tumor")
            break
    #meningioma_tumor
    for features, label in data_set:
        if label == 1:
            axs[0, 1].imshow(features, cmap = "gray")
            axs[0, 1].set_title("meningioma_tumor")
            break
    #no_tumor
    for features, label in data_set:
        if label == 2:
            axs[1, 0].imshow(features, cmap = "gray")
            axs[1, 0].set_title("no_tumor")
            break
    #pituitary_tumor
    for features, label in data_set:
        if label == 3:
            axs[1, 1].imshow(features, cmap = "gray")
            axs[1, 1].set_title("pituitary_tumor")
            break
    plt.show()


#We are going to call the show_image function to show one image from each class in the training data and testing data
show_image(train = True)
show_image(train = False)


In [ ]:
#DATA PREPROCESSING
###############################################

#We are going to shuffle the training data and testing data
#We are going to separate the features and the labels for the training data and testing data
#We are going to split the training data into training data and validation data
#We are going to normalize the training, validation and testing data and one-hot encode the labels

############################################################################################################
# - Shuffle the training data and testing data

#We are going to shuffle the training data
random.shuffle(training_data)
#We are going to shuffle the testing data
random.shuffle(testing_data)

############################################################################################################
# - Separate the features and the labels for the training data and testing data

#We are going to separate the features and the labels for the training data
x_train = []
y_train = []
for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
#We are going to separate the features and the labels for the testing data
x_test = []
y_test = []
for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)

############################################################################################################
# - Split the training data into training data and validation data

VAL_RATIO = 0.2
val_size = int(len(x_train) * VAL_RATIO)
x_val = x_train[:val_size]
y_val = y_train[:val_size]
x_train = x_train[val_size:]
y_train = y_train[val_size:]

############################################################################################################
# - Normalize the training, validation and testing data and one-hot encode the labels

#We are going to normalize the training data by dividing it by 255
x_train = np.array(x_train) / 255
#We are going to normalize the validation data
x_val = np.array(x_val) / 255
#We are going to normalize the testing data
x_test = np.array(x_test) / 255

#We are going to convert the labels to one-hot vectors
y_train = to_categorical(y_train, num_classes = 4)
y_val = to_categorical(y_val, num_classes = 4)
y_test = to_categorical(y_test, num_classes = 4)


In [ ]:
    epochs = {{choice([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])}}
    #We are going to use hyperas to find the best batch size
    batch_size = {{choice([16, 32, 64, 128, 256])}}
    #We are going to use hyperas to find the best learning rate
    learning_rate = {{uniform(0, 1)}}
    #We are going to use hyperas to find the best optimizer
    optimizer = {{choice(['adam', 'rmsprop', 'sgd'])}}
    #We are going to use hyperas to find the best activation function
    activation = {{choice(['relu', 'elu', 'selu', 'tanh', 'sigmoid'])}}
    #We are going to use hyperas to find the best number of neurons in the hidden layer
    neurons = {{choice([16, 32, 64, 128, 256, 512, 1024])}}
    #We are going to use hyperas to find the best dropout rate
    dropout_rate = {{uniform(0, 1)}}
    #We are going to use hyperas to find the best number of layers
    num_layers = {{choice([1, 2, 3, 4, 5])}}
    #We are going to use hyperas to find the best number of neurons in each layer
    neurons_1 = {{choice([16, 32, 64, 128, 256, 512, 1024])}}
    neurons_2 = {{choice([16, 32, 64, 128, 256, 512, 1024])}}
    neurons_3 = {{choice([16, 32, 64, 128, 256, 512, 1024])}}
    neurons_4 = {{choice([16, 32, 64, 128, 256, 512, 1024])}}
    neurons_5 = {{choice([16, 32, 64, 128, 256, 512, 1024])}}
    #We are going to use hyperas to find the best batch normalization
    batch_normalization = {{choice([True, False])}}
    #We are going to use hyperas to find the best kernel initializer
    kernel_initializer = {{choice(['glorot_uniform', 'he_uniform', 'lecun_uniform', 'glorot_normal', 'he_normal', 'lecun_normal'])}}
    #We are going to use hyperas to find the best kernel regularizer
    kernel_regularizer = {{choice(['l1', 'l2', 'l1_l2'])}}
    #We are going to use hyperas to find the best bias regularizer
    bias_regularizer = {{choice(['l1', 'l2', 'l1_l2'])}}

In [ ]:
#FIRST MODEL: A SIMPLE NEURAL NETWORK
###############################################

#We are going to build a simple neural network model
#We are going to train the model
#We are going to evaluate the model

############################################################################################################
# - Build a simple neural network model

#We are going to build a simple neural network model
model = Sequential()
model.add(Flatten(input_shape = (IMG_SIZE, IMG_SIZE)))
model.add(Dense(256, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(4, activation = "softmax"))
model.summary()

############################################################################################################
# - Train the model

#We are going to compile the model
model.compile(optimizer = Adam(learning_rate = 0.0005), loss = "categorical_crossentropy", metrics = ["accuracy"])
#We are going to train the model
history = model.fit(x_train, y_train, epochs = 20, batch_size = 32, validation_data = (x_val, y_val))

############################################################################################################
# - Plot the training and validation accuracy and loss

#We are going to plot the training and validation accuracy and loss
def plot_accuracy_loss(history):
    plt.figure(figsize = (10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label = "Training Accuracy")
    plt.plot(history.history["val_accuracy"], label = "Validation Accuracy")
    plt.legend()
    plt.title("Accuracy")
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label = "Training Loss")
    plt.plot(history.history["val_loss"], label = "Validation Loss")
    plt.legend()
    plt.title("Loss")
    plt.show()


#We are going to call the plot_accuracy_loss function to plot the training and validation accuracy and loss
plot_accuracy_loss(history)




In [ ]:
############################################################################################################
# - Train the model on the training data and validation data

#We are going to concatenate the training data and validation data
x_train_val = np.concatenate((x_train, x_val), axis = 0)
y_train_val = np.concatenate((y_train, y_val), axis = 0)

#We are going to train the model on the training data and validation data
history = model.fit(x_train_val, y_train_val, epochs = 13, verbose = 0)

############################################################################################################
# - Evaluate the model on the testing data

#We are going to evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy: " + str(test_accuracy))





In [ ]:
############################################################################################################
# - Save the model

#We are going to save the model
model.save("model_ANN.h5")


In [ ]:
# SECOND MODEL: A CONVOLUTIONAL NEURAL NETWORK
#####################################################

#We are going to build a convolutional neural network model
#We are going to train the model
#We are going to evaluate the model

############################################################################################################
# - Build a convolutional neural network model

#We are going to build a convolutional neural network model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(4, activation = "softmax"))
model.summary()

In [ ]:
############################################################################################################
# - Load the data using ImageDataGenerator

CATEGORIES = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
TRAINING_DIR = "H:/Desktop/Machine Learning/CS584-TumorsBrain/data/Training/"
TESTING_DIR = "H:/Desktop/Machine Learning/CS584-TumorsBrain/data/Testing/"
IMG_SIZE = 256
WITH_DATA_AUGMENTATION = False

#We are going to load the data using ImageDataGenerator
if WITH_DATA_AUGMENTATION:
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                        rotation_range = 40,
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True,
                                        fill_mode = "nearest",
                                        validation_split = 0.2)
else:
    train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
val_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255)


#We are going to load the training data
training_set = train_datagen.flow_from_directory(TRAINING_DIR
                                                , target_size = (IMG_SIZE, IMG_SIZE)
                                                , batch_size = 32
                                                , class_mode = "categorical"
                                                , subset = "training"
                                                , shuffle = True
                                                , seed = 42)

#We are going to load the validation data
validation_set = val_datagen.flow_from_directory(TRAINING_DIR
                                                , target_size = (IMG_SIZE, IMG_SIZE)
                                                , batch_size = 32
                                                , class_mode = "categorical"
                                                , subset = "validation"
                                                , shuffle = True
                                                , seed = 42)

#We are going to load the testing data
testing_set = test_datagen.flow_from_directory(TESTING_DIR
                                                , target_size = (IMG_SIZE, IMG_SIZE)
                                                , batch_size = 32
                                                , class_mode = "categorical"
                                                , shuffle = False
                                                , seed = 42)




In [ ]:
############################################################################################################
# - Train the model

#We are going to compile the model

#We are going to use the Adam optimizer
#We are going to use the categorical crossentropy loss function
#We are going to use the accuracy metric
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = "categorical_crossentropy", metrics = ["accuracy"])

#We are going to train the model
history = model.fit(training_set, epochs = 1, validation_data = validation_set)

############################################################################################################
# - Evaluate the model

#Plot the training and validation accuracy and loss
def plot_accuracy_loss(history):
    plt.figure(figsize = (15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label = "Training Accuracy")
    plt.plot(history.history["val_accuracy"], label = "Validation Accuracy")
    plt.legend()
    plt.title("Accuracy")
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label = "Training Loss")
    plt.plot(history.history["val_loss"], label = "Validation Loss")
    plt.legend()
    plt.title("Loss")
    plt.show()

#We are going to evaluate the model
test_loss, test_accuracy = model.evaluate(testing_set)
print("Test Accuracy: " + str(test_accuracy))

#We are going to plot the training and validation accuracy and loss
plot_accuracy_loss(history)



In [ ]:
#Heatmap of the activations on a single image
############################################################################################################
# - Load the image

#We are going to load the image and inmport the necessary libraries
from tensorflow.keras.preprocessing import image

img = image.load_img("H:/Desktop/Machine Learning/CS584-TumorsBrain/data/Testing/glioma_tumor/image(1).jpg", target_size = (IMG_SIZE, IMG_SIZE))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

############################################################################################################
#Visualize the activations of the filters

#We are going to visualize the activations of the filters
from tensorflow.keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
activations = activation_model.predict(img)

#We are going to plot the activations
def plot_activations(activations, layer_names):
    images_per_row = 16
    for layer_name, layer_activation in zip(layer_names, activations):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :, col * images_per_row + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype("uint8")
                display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize = (scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect = "auto", cmap = "viridis")

#We are going to plot the activations
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)
plot_activations(activations, layer_names)

#We are going to plot the activations of the filters by initializing a random image










In [ ]:
#visualization of the filters learned by the convolutional layers by finding the input image that maximizes the activation of a given filter

from tensorflow.keras import backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

#We load the model
#model = load_model('cats_and_dogs_small_1.h5')

layer_outputs = [layer.output for layer in model.layers[:8]]

def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    x += 0.5
    x = np.clip(x, 0, 1)
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def generate_pattern(layer_number, filter_index, size=150):
    layer_output = layer_outputs[layer_number]
    loss = K.mean(layer_output[:, :, :, filter_index])
    #We compute the gradient of the loss with respect to the input
    grads = K.gradients(loss, model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    iterate = K.function([model.input], [loss, grads])
    #We initialize the input image with a random noise

    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    step = 0.8
    #We run gradient ascent for 40 steps
    for i in range(65):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
    img = input_img_data[0]
    return deprocess_image(img)

#We plot the first filter of the first convolutional layer

plt.imshow(generate_pattern(5, 7))
plt.show()

#We plot all the filters of the first convolutional layer in black and white

#We plot all the filters of each convolutional layer in black and white (the first convolutional layer has 32 filters, the second 64, the third 128, the fourth 128)

print("GRID FOR THE FIRST CONVOLUTIONAL LAYER")
#make a grid of 8*4 images
fig, ax = plt.subplots(8, 4, figsize=(32, 32))
for i in range(8):
    for j in range(4):
        ax[i, j].imshow(generate_pattern(0, (i+1) * (1+j) - 1), cmap='viridis')
        ax[i, j].axis('off')
plt.show()


print("GRID FOR THE SECOND CONVOLUTIONAL LAYER")
#make a grid of 8*8 images
fig, ax = plt.subplots(8, 8, figsize=(32, 32))
for i in range(8):
    for j in range(8):
        ax[i, j].imshow(generate_pattern(2, (i+1) * (1+j) - 1), cmap='viridis')
        ax[i, j].axis('off')
plt.show()

print("GRID FOR THE THIRD CONVOLUTIONAL LAYER")
#make a grid of 16*8 images
fig, ax = plt.subplots(16, 8, figsize=(32, 32))
for i in range(16):
    for j in range(8):
        ax[i, j].imshow(generate_pattern(4, (i+1) * (1+j) - 1), cmap='viridis')
        ax[i, j].axis('off')
plt.show()

print("GRID FOR THE FOURTH CONVOLUTIONAL LAYER")
#make a grid of 16*8 images
fig, ax = plt.subplots(16, 8, figsize=(32, 32))
for i in range(16):
    for j in range(8):
        ax[i, j].imshow(generate_pattern(6, (i+1) * (1+j) - 1), cmap='viridis')
        ax[i, j].axis('off')
plt.show()